 Logistic regression with words and char n-grams

In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [3]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('./input/train.csv').fillna(' ')
test = pd.read_csv('./input/test.csv').fillna(' ')

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

print('Loaded.')

Loaded.


In [4]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
print('Fitting word vectorizer.')
word_vectorizer.fit(all_text)

print('Transforming train word features.')
train_word_features = word_vectorizer.transform(train_text)
print('Transforming test word features.')
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
print('Fitting char vectorizer.')
char_vectorizer.fit(all_text)

print('Transforming train char features.')
train_char_features = char_vectorizer.transform(train_text)
print('Transforming test char features.')
test_char_features = char_vectorizer.transform(test_text)

print('Features are transformed.')

Fitting word vectorizer.
Transforming train word features.
Transforming test word features.
Fitting char vectorizer.
Transforming train char features.
Transforming test char features.
Features are transformed.


In [5]:
print('Stacking train features.')
train_features = hstack([train_char_features, train_word_features])
print('Stacking test features.')
test_features = hstack([test_char_features, test_word_features])

print('Features are stacked.')

Stacking train features.
Stacking test features.
Features are stacked.


In [6]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

print('Start training.')
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))


Start training.
CV score for class toxic is 0.9783248247081348
CV score for class severe_toxic is 0.9885337617156439
CV score for class obscene is 0.9901305132120924
CV score for class threat is 0.9896541182183816
CV score for class insult is 0.9826306655543111
CV score for class identity_hate is 0.9823772153427436
Total CV score is 0.9852751831252179


In [7]:
submission.to_csv('lr_submission.csv', index=False)